In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import random

plt.rcParams['font.size']=12
plt.rcParams['font.family']='serif'
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False  
plt.rcParams['axes.spines.bottom'] = True     
plt.rcParams["axes.grid"] = True
plt.rcParams['grid.linestyle'] = '-.' 
plt.rcParams['grid.linewidth'] = 0.4

In [2]:
with open('Data/ALL_scenarios.json') as f:
    all_scenarios = json.load(f)

all_scenarios.keys()
OMEGA = 250 # number of scenarios to sample
PI = 1 / OMEGA # probability of each sampled scenario - assumed to be equal

S = len(all_scenarios.keys()) - 1 # number of total scenarios
T = 24 # number of hours

WIND_CAPACITY = 200 #MWh

random.seed(123)

# Sample scenarios without replacement
in_sample_scenarios = random.sample(range(S), 250)

scenarios = {}

# Extract sampled scenarios from dictionary containing all scenarios
for i in range(len(in_sample_scenarios)):
    scenarios[str(i)] = all_scenarios[str(in_sample_scenarios[i])]
    scenarios[str(i)]['Original Index'] = in_sample_scenarios[i]

Get out-of-sample scenarios

In [4]:
# Indices of all scenarios
all_indices = list(range(S))

# Indices of out-of-sample scenarios
out_of_sample_indices = list(set(all_indices) - set(in_sample_scenarios))

# Sample 950 out-of-sample scenarios without replacement
out_sample_indices = random.sample(out_of_sample_indices, 950)
print(out_sample_indices)

# Extract out-of-sample scenarios from the dictionary containing all scenarios
out_sample_scenarios = {}
for i, idx in enumerate(out_sample_indices):
    out_sample_scenarios[str(i)] = all_scenarios[str(idx)]
    out_sample_scenarios[str(i)]['Original Index'] = idx

# Print information
print('Number of out-of-sample scenarios:', len(out_sample_scenarios))

[378, 1226, 756, 1214, 1210, 1306, 330, 803, 1178, 253, 1277, 1706, 863, 521, 1338, 794, 228, 630, 1190, 566, 474, 749, 612, 936, 109, 1179, 199, 460, 1188, 465, 1777, 45, 454, 620, 1327, 534, 210, 1408, 738, 206, 796, 1301, 1260, 321, 1899, 1253, 325, 1844, 1139, 1160, 1719, 1973, 1823, 72, 595, 1675, 1356, 1143, 795, 928, 520, 1180, 305, 1428, 1489, 15, 1865, 1857, 1410, 1427, 1164, 1991, 1634, 1374, 296, 1623, 133, 842, 1082, 845, 1569, 986, 735, 838, 504, 1807, 1053, 802, 1904, 1768, 451, 427, 897, 1893, 155, 973, 876, 1990, 614, 414, 1313, 879, 862, 171, 686, 1918, 1894, 1183, 1418, 1976, 138, 892, 130, 1377, 909, 694, 261, 1093, 1346, 1795, 1660, 755, 564, 1516, 1043, 615, 1030, 1322, 1341, 804, 576, 1622, 1070, 1931, 1350, 637, 786, 121, 430, 298, 51, 1005, 1549, 1168, 1725, 318, 1124, 1254, 108, 1108, 1269, 339, 758, 865, 513, 1403, 82, 1088, 1186, 1924, 1816, 668, 1659, 1361, 36, 746, 1716, 905, 528, 1615, 156, 586, 659, 400, 977, 836, 999, 1971, 1423, 34, 1169, 1580, 392, 462

Import the optimal day-ahead dispatch values

In [6]:
optimal_DA_bids = pd.read_csv('Data/optimal_DA_bids.csv', index_col=0)

optimal_DA_oneprice = optimal_DA_bids['One-price Bids [MW]'].values
optimal_DA_twoprice = optimal_DA_bids['Two-price Bids [MW]'].values

Calculate out-of-sample performance **(one-price)**

In [5]:
out_sample_scenarios['0'].keys()

dict_keys(['Wind Power [MW]', 'Spot Price [EUR/MWh]', 'System Balance State', 'Original Index'])

In [11]:
# Calculate payments or earnings for out-of-sample scenarios
total_payment_earnings = 0

for scenario_idx in out_sample_scenarios:
    scenario = out_sample_scenarios[scenario_idx]

    x = scenario['Spot Price [EUR/MWh]'] * optimal_DA_oneprice

    realized_imbalance = np.array(scenario['Wind Power [MW]']) - optimal_DA_oneprice

    price_coefficient = 1.2 * np.array(scenario['System Balance State']) + 0.9 * (1 - np.array(scenario['System Balance State']))
    y = realized_imbalance * price_coefficient * scenario['Spot Price [EUR/MWh]']

    payment_earnings = x + y
    
    total_payment_earnings += np.sum(payment_earnings)

# Calculate average payment or earnings over out-of-sample scenarios
average_payment_earnings_one_price = total_payment_earnings / len(out_sample_scenarios)

print(average_payment_earnings_one_price)

181065.23900215412


Calculate out-of-sample performance **(two-price)**

In [15]:
# Calculate payments or earnings for out-of-sample scenarios
total_payment_earnings = 0

for scenario_idx in out_sample_scenarios:
    scenario = out_sample_scenarios[scenario_idx]

    x = scenario['Spot Price [EUR/MWh]'] * optimal_DA_twoprice

    realized_imbalance = np.array(scenario['Wind Power [MW]']) - optimal_DA_twoprice

    price_coefficients = np.zeros(T)

    for t in range(T):
        if (realized_imbalance[t] > 0) and (scenario['System Balance State'][t] == 0):
            price_coefficients[t] = 0.9
        elif (realized_imbalance[t] > 0) and (scenario['System Balance State'][t] == 1):
            price_coefficients[t] = 1.0
        elif (realized_imbalance[t] < 0) and (scenario['System Balance State'][t] == 0):
            price_coefficients[t] = -1.0
        elif (realized_imbalance[t] < 0) and (scenario['System Balance State'][t] == 1):
            price_coefficients[t] = -1.2
    

    y = np.abs(realized_imbalance) * price_coefficients * scenario['Spot Price [EUR/MWh]']

    payment_earnings = x + y
    
    total_payment_earnings += np.sum(payment_earnings)

# Calculate average payment or earnings over out-of-sample scenarios
average_payment_earnings_two_price = total_payment_earnings / len(out_sample_scenarios)

print(average_payment_earnings_two_price)

164037.96700124204
